<a href="https://colab.research.google.com/github/sofiaviale-hue/COOvalidator/blob/main/Online_e_COO_Authenticator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 0. PREREQUISITES AND IMPORTS ---

# Install necessary libraries for PDF processing, QR code reading, web scraping, and Gradio
!pip install gradio
!apt -y update >/dev/null
!apt -y install poppler-utils libzbar0 tesseract-ocr >/dev/null
!pip -q install pdf2image pyzbar pillow pytesseract opencv-python pandas beautifulsoup4 requests pypdf
!pip -q install playwright nest_asyncio
!python -m playwright install chromium

import gradio as gr
import os, re, io, base64, time, asyncio, requests, numpy as np
from urllib.parse import urlparse, parse_qs, unquote, urljoin
from pdf2image import convert_from_path
from PIL import Image, ImageOps
import pytesseract, cv2
from pyzbar.pyzbar import decode as zbar_decode
from bs4 import BeautifulSoup
from pypdf import PdfReader
import pandas as pd
import nest_asyncio
from playwright.async_api import async_playwright
import tempfile
from collections import defaultdict
from datetime import datetime

# Apply nest_asyncio to allow nested event loops (needed for Playwright in Colab)
nest_asyncio.apply()

# --- 1. GLOBAL CONFIGURATION & CORE HELPER FUNCTIONS ---

DPI = 600
UA  = {"User-Agent": "Mozilla/5.0 (COO Validator; Colab)"}
CONNECT_T, READ_T = 5, 10
ALLOWED_HOSTS = {
    "ecosys.gov.vn", "www.ecosys.gov.vn",
    "ska.kemendag.go.id", "www.ska.kemendag.go.id"
}
FORCE_SKAS = {"ska.kemendag.go.id", "www.ska.kemendag.go.id"}

_RENDER, _URL2REF, _SES = {}, {}, None
URL_RX = re.compile(r"https?://[A-Za-z0-9\.\-_/%?&=+#~]+", re.I)

# PDF Extraction Patterns
LABELS = ("REFERENCE NO","REFERENCE NUMBER","CERTIFICATE NO","CERTIFICATE NUMBER",
          "REF. NO","REF NO","C/O NO","C/O NUMBER")
PAT_VN = re.compile(r"\b([A-Z]{2,3})\s*-\s*([A-Z]{2,3})\s*([0-9/\s\-]{5,})\b")
PAT_ID = re.compile(r"\b(\d{4,}\s*/\s*[A-Z]{2,}\s*/\s*\d{4})\b")
PAT_GENERIC = re.compile(r"(?=.*[A-Z])(?=.*\d)(?=.*[/-])[A-Z0-9/-]{6,}")
PARAMS = {
    "ecosys.gov.vn": ["CertificateNumber","certificateNumber","ReferenceNo","RefNo","CertNo"],
    "_default":      ["CertificateNumber","ReferenceNo","RefNo","CertNo","No","no"]
}

# --- Image Handling and Core Logic Functions ---

def get_base64_uri(image_path, file_type='png'):
    """Reads a local image and converts it to a Base64 data URI."""
    try:
        mime_type = 'jpeg' if file_type.lower() in ['jpg', 'jpeg'] else file_type.lower()
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/{mime_type};base64,{encoded_string}"
    except FileNotFoundError:
        print(f"Image not found: {image_path}. Using fallback color.")
        return None

def _ses():
    global _SES
    if _SES is None:
        s = requests.Session()
        s.headers.update(UA)
        _SES = s
    return _SES

def render_first_page(path: str, dpi=DPI):
    key = (path, dpi)
    if key in _RENDER: return _RENDER[key]
    img = convert_from_path(path, dpi=dpi, first_page=1, last_page=1)[0]
    _RENDER[key] = img
    return img

def norm_ref(s: str) -> str:
    s = (s or "").upper().replace("—","-").replace("–","-")
    s = re.sub(r"\s+", "", s)
    s = re.sub(r"[^A-Z0-9/\-]", "", s)
    return s

def looks_like_ref(s: str) -> bool:
    s = norm_ref(s)
    if not s or len(s) < 8: return False
    return bool(re.search(r"\d", s)) and ("/" in s or "-" in s)

def _pdf_text_first_page(path: str) -> str:
    try:
        r = PdfReader(path)
        if not r.pages: return ""
        return r.pages[0].extract_text() or ""
    except Exception:
        return ""

def _ocr_lines(img: Image.Image):
    d = pytesseract.image_to_data(ImageOps.grayscale(img), lang="eng",
                                  output_type=pytesseract.Output.DICT,
                                  config="--oem 3 --psm 6")
    toks = []
    for i in range(len(d["text"])):
        t = (d["text"][i] or "").strip()
        if not t:
            continue
        toks.append(dict(
            text=t.upper().replace("N0","NO"),
            left=d["left"][i], top=d["top"][i], w=d["width"][i], h=d["height"][i],
            line=(d["page_num"][i], d["block_num"][i], d["par_num"][i], d["line_num"][i])
        ))
    from collections import defaultdict
    by_line = defaultdict(list)
    for t in toks: by_line[t["line"]].append(t)
    return [(lid, sorted(arr, key=lambda x: x["left"])) for lid, arr in sorted(by_line.items())]

def _pick_best(cands):
    scored = []
    for c in cands:
        s = norm_ref(c)
        if not s: continue
        if not re.search(r"[A-Z]", s): continue
        if not ("/" in s or "-" in s): continue
        score = 0
        if PAT_VN.search(s): score += 5
        if PAT_ID.search(s): score += 5
        if s.startswith(("VN-","ID-")): score += 2
        if "/" in s: score += 1
        if "-" in s: score += 1
        scored.append((score, s))
    scored.sort(reverse=True)
    return scored[0][1] if scored else ""

def extract_ref_from_pdf(pdf_path: str) -> str:
    txt = _pdf_text_first_page(pdf_path)
    if txt:
        U = txt.upper().replace("N0","NO")
        m = re.search(r"(REFERENCE|CERTIFICATE)\s*(NO|NUMBER)\s*[:\-]?.?\s*([A-Z0-9/\-\s]{6,})", U)
        if m:
            win = m.group(3)
            cands = []
            for mm in (PAT_VN.search(win), PAT_ID.search(win), PAT_GENERIC.search(win)):
                if mm: cands.append(mm.group(0) if mm.re is not PAT_VN else f"{mm.group(1)}-{mm.group(2)}{mm.group(3)}")
            best = _pick_best(cands)
            if best: return best
        cands = []
        for mm in PAT_VN.finditer(U):  cands.append(f"{mm.group(1)}-{mm.group(2)}{mm.group(3)}")
        for mm in PAT_ID.finditer(U):  cands.append(mm.group(1))
        for mm in PAT_GENERIC.finditer(U): cands.append(mm.group(0))
        best = _pick_best(cands)
        if best: return best

    try:
        page = render_first_page(pdf_path, DPI)
    except Exception:
        return ""
    lines = _ocr_lines(page)

    all_cands = []
    for idx, (_, arr) in enumerate(lines):
        line_txt = " ".join(a["text"] for a in arr)
        if not any(lbl in line_txt for lbl in LABELS):
            continue
        rightmost = max((a["left"]+a["w"] for a in arr if any(lbl in a["text"] for lbl in LABELS)), default=0)
        same = " ".join(a["text"] for a in arr if a["left"] > rightmost)
        nxt  = " ".join(a["text"] for a in (lines[idx+1][1] if idx+1 < len(lines) else []))
        blob = " ".join(x for x in (same, nxt) if x)
        if blob:
            for mm in PAT_VN.finditer(blob):  all_cands.append(f"{mm.group(1)}-{mm.group(2)}{mm.group(3)}")
            for mm in PAT_ID.finditer(blob):  all_cands.append(mm.group(1))
            for mm in PAT_GENERIC.finditer(blob): all_cands.append(mm.group(0))
    best = _pick_best(all_cands)
    if best: return best

    text = pytesseract.image_to_string(ImageOps.grayscale(page), lang="eng",
                                       config="--oem 3 --psm 6").upper().replace("N0","NO")
    cands = []
    for mm in PAT_VN.finditer(text):  cands.append(f"{mm.group(1)}-{mm.group(2)}{mm.group(3)}")
    for mm in PAT_ID.finditer(text):  cands.append(mm.group(1))
    for mm in PAT_GENERIC.finditer(text): cands.append(mm.group(0))
    best = _pick_best(cands)
    return best or ""

def _decode_all(img: Image.Image): # ... (retained logic)
    texts = []
    try:
        for d in zbar_decode(img):
            if d.type == "QRCODE":
                texts.append(d.data.decode("utf-8", errors="ignore").strip())
    except Exception:
        pass
    try:
        bgr = cv2.cvtColor(np.array(img.convert("RGB")), cv2.COLOR_RGB2BGR)
        det = cv2.QRCodeDetector()
        data, pts, _ = det.detectAndDecode(bgr)
        if data: texts.append(data.strip())
    except Exception:
        pass
    out, seen = [], set()
    for t in texts:
        if t not in seen:
            out.append(t); seen.add(t)
    return out

def _prep_variants(pil_img: Image.Image): # ... (retained logic)
    out = []
    for rot in (0, 90, 180, 270):
        base = pil_img if rot == 0 else pil_img.rotate(rot, expand=True)
        g = ImageOps.grayscale(base)
        arr = np.array(g)
        out.append(g)
        _, otsu = cv2.threshold(arr, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        out.append(Image.fromarray(otsu))
        adp = cv2.adaptiveThreshold(arr, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, 2)
        out.append(Image.fromarray(adp))
        blur = cv2.medianBlur(arr, 3)
        adp2 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                     cv2.THRESH_BINARY, 41, 4)
        out.append(Image.fromarray(adp2))
        out.append(ImageOps.invert(Image.fromarray(adp2)))
    return out

def _corner_crops(pil_img: Image.Image, frac=0.40): # ... (retained logic)
    W, H = pil_img.width, pil_img.height
    w, h = int(W*frac), int(H*frac)
    return [
        pil_img.crop((0, 0, w, h)),
        pil_img.crop((W-w, 0, W, h)),
        pil_img.crop((0, H-h, w, H)),
        pil_img.crop((W-w, H-h, W, H)),
    ]

def _is_valid_url(u: str) -> bool: # ... (retained logic)
    try:
        up = urlparse(u.strip())
        if up.scheme not in ("http","https"): return False
        if not up.netloc: return False
        host = up.netloc.lower()
        return any(host.endswith(h) for h in ALLOWED_HOSTS)
    except Exception:
        return False

def extract_qr_url_from_pdf(pdf_path: str): # ... (retained logic)
    for dpi in (600, 800, 500):
        try:
            page = convert_from_path(pdf_path, dpi=dpi, first_page=1, last_page=1)[0]
        except Exception:
            continue

        for v in _prep_variants(page):
            for s in _decode_all(v):
                if _is_valid_url(s): return s

        for corner in _corner_crops(page, frac=0.40):
            for v in _prep_variants(corner):
                for s in _decode_all(v):
                    if _is_valid_url(s): return s

        text = pytesseract.image_to_string(ImageOps.grayscale(page), lang="eng", config="--oem 3 --psm 6")
        for u in URL_RX.findall(text):
            if _is_valid_url(u): return u.strip()

    try:
        page = render_first_page(pdf_path, DPI)
        text = pytesseract.image_to_string(ImageOps.grayscale(page), lang="eng", config="--oem 3 --psm 6")
        for u in URL_RX.findall(text):
            if _is_valid_url(u): return u.strip()
    except Exception:
        pass
    return None

def extract_ref_from_url(url: str) -> str: # ... (retained logic)
    if not url: return ""
    host = urlparse(url).netloc.lower()
    if host in FORCE_SKAS:
        return ""
    up = urlparse(url); q = parse_qs(up.query)
    keys = PARAMS.get(host, PARAMS["_default"])
    for k in keys:
        if q.get(k):
            v = norm_ref(unquote(q[k][0]))
            return v if looks_like_ref(v) else ""
    tail = unquote(up.path.strip("/").split("/")[-1]).upper()
    m = PAT_GENERIC.search(tail)
    if m:
        v = norm_ref(m.group(0))
        return v if looks_like_ref(v) else ""
    m = PAT_GENERIC.search(unquote(url).upper())
    if m:
        v = norm_ref(m.group(0))
        return v if looks_like_ref(v) else ""
    return ""

def fetch_ref_requests(url: str) -> str: # ... (retained logic)
    if not url: return ""
    if url in _URL2REF: return _URL2REF[url]
    s = _ses()
    try:
        with tempfile.NamedTemporaryFile(suffix=".pdf", delete=True) as tmp_file:
            r = s.get(url, headers=UA, timeout=(CONNECT_T, READ_T), allow_redirects=True)
            final = r.url or url

            ref = extract_ref_from_url(final)
            if looks_like_ref(ref): _URL2REF[url] = ref; return ref

            ctype = (r.headers.get("Content-Type") or "").lower()
            body  = r.content

            if "application/pdf" in ctype or body[:4]==b"%PDF" or final.lower().endswith(".pdf"):
                tmp_file.write(body)
                tmp_file.flush()
                ref = extract_ref_from_pdf(tmp_file.name)
                return ref if looks_like_ref(ref) else ""

            try:
                txt = body.decode("utf-8", errors="ignore").upper().replace("N0","NO")
            except Exception:
                txt = (body or b"").decode("latin-1", errors="ignore").upper().replace("N0","NO")

            m = PAT_ID.search(txt) or PAT_VN.search(txt)
            if m:
                ref = norm_ref(m.group(1) if m.re is PAT_ID else f"{m.group(1)}-{m.group(2)}{m.group(3)}")
                if looks_like_ref(ref): _URL2REF[url] = ref; return ref

            return ""
    except Exception:
        return ""

def refs_equal(a: str, b: str) -> bool: # ... (retained logic)
    A, B = norm_ref(a), norm_ref(b)
    if not A or not B: return False
    if A == B: return True
    trans = str.maketrans({"I":"J","J":"I","O":"0","0":"O","1":"I","L":"1"})
    if A.translate(trans) == B or B.translate(trans) == A: return True
    pA, pB = A[:6], B[:6]
    mism = sum(x!=y for x,y in zip(pA,pB)) + abs(len(pA)-len(pB))
    return mism <= 1 and A[6:] == B[6:]

# Playwright placeholders (not used but included for completeness)
def _ocr_image_bytes_for_ref(img_bytes: bytes) -> str: return ""
async def _scan_all_frames(page, ctx) -> str: return ""
async def _fetch_ref_playwright_async(url: str) -> str: return ""
def fetch_ref_playwright(url: str) -> str: return ""


# --- 2. MAIN VALIDATION LOGIC ---

def check_coo(pdf_path: str) -> dict:
    """Main logic to extract and compare references."""
    _URL2REF.clear()

    # 1. Extract from PDF/OCR
    pdf_ref = extract_ref_from_pdf(pdf_path)

    # 2. Extract QR Code URL
    qr_url  = extract_qr_url_from_pdf(pdf_path)

    host = urlparse(qr_url).netloc.lower() if qr_url else ""
    url_ref, how = "", ""

    if qr_url:
        # 3a. Attempt URL parameter extraction (fastest)
        url_ref = extract_ref_from_url(qr_url)
        if looks_like_ref(url_ref): how = "url_params"

        # 3b. Attempt fast HTTP fetch/redirect/PDF sniff
        if not url_ref:
            url_ref = fetch_ref_requests(qr_url)
            if looks_like_ref(url_ref): how = "requests_fetch"

    # --- Final Status Check ---
    if not pdf_ref:
        return {"status": "Error", "reason": "No Reference No found on PDF",
                "PDF_reference": None, "URL_reference": url_ref or None, "qr_url": qr_url, "host": host, "how": how}
    if not qr_url:
        return {"status": "Error", "reason": "No QR code found or undecodable",
                "PDF_reference": pdf_ref, "URL_reference": url_ref or None, "qr_url": None, "host": host, "how": how}
    if not url_ref:
        return {"status": "Error", "reason": "No Reference found at QR target",
                "PDF_reference": pdf_ref, "URL_reference": None, "qr_url": qr_url, "host": host, "how": how}

    status = "Valid" if refs_equal(pdf_ref, url_ref) else "FraudSuspected"
    reason = "Reference match" if status == "Valid" else "Reference mismatch"
    return {"status": status, "reason": reason,
            "PDF_reference": pdf_ref, "URL_reference": url_ref,
            "qr_url": qr_url, "host": host, "how": how}


# --- 3. CSV LOGGING SETUP ---

CSV_LOG_FILE = 'coo_assessment_history.csv'

def log_to_csv(result: dict, filename: str):
    """Appends the validation result to a local CSV file."""

    row = {
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Filename": filename,
        "Status": result.get("status", "N/A"),
        "Reason": result.get("reason", "N/A"),
        "PDF Reference": result.get("PDF_reference", "N/A"),
        "URL Reference": result.get("URL_reference", "N/A"),
        "QR URL": result.get("qr_url", "N/A"),
        "Host": result.get("host", "N/A"),
        "Extraction Method": result.get("how", "N/A"),
    }

    file_exists = os.path.exists(CSV_LOG_FILE)

    try:
        df_new = pd.DataFrame([row])
        df_new.to_csv(
            CSV_LOG_FILE,
            mode='a',
            header=not file_exists,
            index=False
        )
        print(f"Result logged successfully to {CSV_LOG_FILE}")
        return True
    except Exception as e:
        print(f"CSV Logging Error: {e}")
        return False


# --- 4. GRADIO INTERFACE LOGIC & VISUALS SETUP ---

# Define Image Paths (Ensure these files are uploaded to Colab)
LOCAL_LOGO_PATH = 'logo.png'
LOCAL_BACKGROUND_PATH = 'background.jpg'

# Generate Base64 URIs for embedding
LOGO_BASE64 = get_base64_uri(LOCAL_LOGO_PATH, file_type='png')
# NOTE: Background image data is generated but we rely on solid color for stability
BACKGROUND_BASE64 = get_base64_uri(LOCAL_BACKGROUND_PATH, file_type='jpg')

def validation_interface(file):
    """Gradio wrapper function to process the uploaded file."""

    if file is None:
        return "Please upload a Certificate of Origin (COO) document to begin the assessment."

    pdf_path = file.name
    filename = os.path.basename(pdf_path)

    # Yield the 'Running Assessment' message immediately to make the spinner appear
    yield f"## 🏃 Running Assessment... Please wait."

    print(f"Processing uploaded file: {pdf_path}")

    # This is the actual processing call
    result = check_coo(pdf_path)

    # Log the result
    log_success = log_to_csv(result, filename)
    log_message = f" (Log Status: {'Success' if log_success else 'FAILED'})"

    # Format the final result (no bolding)
    status_emoji = "✓" if result["status"] == "Valid" else ("✗" if result["status"] == "FraudSuspected" else "⚠️")

    output = f"## {status_emoji} Validation Assessment: {result['status']}{log_message}\n"
    output += f"--- \n"
    output += f"Reason: {result['reason']}\n\n"
    output += f"1. Document Reference Number (PDF/OCR):\n- Reference: {result['PDF_reference'] or 'N/A'}\n\n"
    output += f"2. Online Verification (via QR Code):\n"
    output += f"- QR URL Found: {result['qr_url'] or 'N/A'}\n"
    output += f"- Host: {result['host'] or 'N/A'}\n"
    f"- Online Reference: {result['URL_reference'] or 'N/A'}\n"
    output += f"- Extraction Method: {result['how'] or 'N/A (No reference found)'}\n"

    if result['status'] == 'FraudSuspected':
        output += "\n---\n⚠️ ALERT: REFERENCE MISMATCH DETECTED. MANUAL REVIEW STRONGLY RECOMMENDED."
    elif result['status'] == 'Valid':
        output += "\n---\nSUCCESS: Reference numbers match. Document appears valid."

    # Yield the final result to update the UI
    yield output


# Construct the Title based on Base64 image status
TITLE_HTML = ""
if LOGO_BASE64:
    # Embed the logo image to the left of the title text
    TITLE_HTML = f'<img src="{LOGO_BASE64}" style="height: 35px; vertical-align: middle; margin-right: 10px;"> Electronic COO Online Validator'
else:
    # Fallback to simple text/emoji if logo file not found
    TITLE_HTML = "🏃 Electronic COO Online Validator"


# Construct the custom CSS including the button styles
custom_css = f"""
:root {{
    --font: 'Diatype', 'Inter', 'Open Sans', sans-serif !important;
    --background-fill-primary: #e1dbcd !important;
    --background-fill-secondary: #f0f0e8 !important;
    --text-color-subdued: #444444 !important;
    --text-color-body: #000000 !important;
    --color-text: #000000 !important;
    --font-weight-bold: 400 !important;
    --border-color-primary: #a0a0a0 !important;

    /* 🌟 FIX: Override Primary Button Colors to Match Secondary (White Look) */
    --button-primary-background-fill: #ffffff !important;
    --button-primary-background-fill-hover: #f0f0e8 !important;
    --button-primary-text-color: #000000 !important;
    --button-primary-border-color: var(--color-text) !important;
}}

/* Set the overall body background (Rely on solid color) */
body {{
    background-color: var(--background-fill-primary) !important;
    /* BACKGROUND IMAGE REMOVED FOR STABILITY */
}}

/* Centering and sizing the main content area */
.gradio-container {{
    display: flex;
    justify-content: center;
    align-items: flex-start;
    min-height: 100vh;
    padding-top: 5vh;
    padding-bottom: 5vh;
}}

#interface-root {{
    max-width: 800px;
    width: 100%;
    background-color: var(--background-fill-primary) !important;
    box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
}}

/* Applying Diatype and new colors */
body, .gradio-container, .svelte-16b7l0s, button, input, textarea, select, h1, h2, h3, h4, strong {{
    font-family: var(--font);
    color: var(--color-text) !important;
    font-weight: 400 !important;
}}

/* Validation Result Box Styling */
#validation-result {{
    border: 1px solid var(--border-color-primary);
    padding: 15px;
    background-color: var(--background-fill-secondary);
}}

#validation-result strong {{
    font-weight: 400 !important;
}}

/* Ensure both buttons have the required border style */
button {{
    border: 1px solid var(--button-primary-border-color) !important;
    border-radius: 0 !important;
}}

/* Hide the default 'Flag' and 'Flagged' elements entirely as logging is external */
div.flagging-container, .tabs > .tab-button:last-child {{
    display: none !important;
}}
"""

# --- 5. GRADIO SETUP ---

# Define Components
file_input = gr.File(
    label="Upload Certificate of Origin (COO) Document (PDF)",
    file_types=[".pdf"]
)
output_text = gr.Markdown(
    label="Validation Assessment Result",
    elem_id="validation-result"
)

# Create the Gradio Interface
iface = gr.Interface(
    fn=validation_interface,
    inputs=file_input,
    outputs=output_text,
    title=TITLE_HTML, # Use the dynamic HTML title with embedded logo
    description=f"Upload a single e-COO PDF to automatically extract its reference number, decode its QR code, and compare the references with the online source. Results are logged to the local file: **{CSV_LOG_FILE}**",

    allow_flagging="never",

    theme=gr.themes.Monochrome(
        primary_hue="neutral",
        secondary_hue="neutral",
    ).set(
        body_background_fill='#e1dbcd',
        body_text_color='black',
        button_secondary_background_fill='white',
        button_secondary_text_color='black'
    ),
    css=custom_css
)

# Launch the Interface
iface.launch(
    debug=True
)




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Please install them with the following command:      ║
║                                                      ║
║     playwright install-deps                          ║
║                                                      ║
║ Alternatively, use apt:                              ║
║     apt-get install libatk1.0-0\                     ║
║         libatk-bridge2.0-0\                          ║
║         libatspi2.0-0\                               ║
║         libxcomposite1                               ║
║                                                      ║
║ <3 Playwright Team                                   ║
╚════════════════════════════

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e9af035ed5167f337a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Processing uploaded file: /tmp/gradio/20585375cf3206a07ef9f0c2d027c12d8a88ab518d1911699f71bf3d0243155d/VN-CA CPTPP COO.pdf
Result logged successfully to coo_assessment_history.csv
